In [1]:
import requests, xmltodict, pandas as pd
from pandas import json_normalize
from zeep import Client, helpers
from datetime import datetime
import ast

In [2]:
def listar_ops(wsdl_url):
    c = Client(wsdl=wsdl_url)
    for svc in c.wsdl.services.values():
        for port in svc.ports.values():
            binding = port.binding
            for name, op in binding._operations.items():
                # Nombre de la operación
                args = []
                if op.input and op.input.body and op.input.body.type:
                    args = [elt[0] for elt in op.input.body.type.elements]  # nombres de parámetros
                print(f"{name}({', '.join(args)})")

# Servicio raíz (Cámara)
#listar_ops("https://opendata.camara.cl/wscamaradiputados.asmx?WSDL")
# Servicio Legislativo
listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
# Servicio Diputado / Comisión / Común (si los necesitas)
#listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSDiputado.asmx?WSDL")
#listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSComision.asmx?WSDL")
#listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSComun.asmx?WSDL")


retornarPeriodosLegislativos()
retornarPeriodoLegislativoActual()
retornarLegislaturas()
retornarLegislaturaActual()
retornarMensajesXAnno(prmAnno)
retornarMocionesXAnno(prmAnno)
retornarMaterias()
retornarTramitesConstitucionales()
retornarTramitesReglamentarios()
retornarVotacionesXProyectoLey(prmNumeroBoletin)
retornarVotacionesXAnno(prmAnno)
retornarVotacionDetalle(prmVotacionId)
retornarProyectoLey(prmNumeroBoletin)
retornarProyectosLeyxNumeroLey(prmNumeroLey)
retornarPeriodosLegislativos()
retornarPeriodoLegislativoActual()
retornarLegislaturas()
retornarLegislaturaActual()
retornarMensajesXAnno(prmAnno)
retornarMocionesXAnno(prmAnno)
retornarMaterias()
retornarTramitesConstitucionales()
retornarTramitesReglamentarios()
retornarVotacionesXProyectoLey(prmNumeroBoletin)
retornarVotacionesXAnno(prmAnno)
retornarVotacionDetalle(prmVotacionId)
retornarProyectoLey(prmNumeroBoletin)
retornarProyectosLeyxNumeroLey(prmNumeroLey)
retornarPeriodosLegislativos()
retornarPeriodoLegislativoAct

In [3]:
c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
res = c.service.retornarPeriodosLegislativos()
print(res)

[{
    'Id': 1,
    'Nombre': '1990-1994',
    'FechaInicio': datetime.datetime(1990, 3, 11, 0, 0),
    'FechaTermino': datetime.datetime(1994, 3, 10, 0, 0),
    'Legislaturas': {
        'Legislatura': [
            {
                'Id': 3,
                'Numero': 319,
                'FechaInicio': datetime.datetime(1990, 3, 11, 0, 0),
                'FechaTermino': datetime.datetime(1990, 5, 20, 23, 59, 59),
                'Tipo': {
                    '_value_1': 'Extraordinaria',
                    'Valor': 2
                }
            },
            {
                'Id': 4,
                'Numero': 320,
                'FechaInicio': datetime.datetime(1990, 5, 21, 0, 0),
                'FechaTermino': datetime.datetime(1990, 9, 17, 23, 59, 59),
                'Tipo': {
                    '_value_1': 'Ordinaria',
                    'Valor': 1
                }
            },
            {
                'Id': 5,
                'Numero': 321,
                'Fec

In [4]:
def create_instance(r):
    return {
        "id" : r["Id"],
        "nombre" : r["Nombre"],
        "fecha_inicio" : r["FechaInicio"],
        "fecha_termino" : r["FechaTermino"]
    }

df_periodos_legislativos = pd.DataFrame(list(map(create_instance, res)))

In [2]:
import pandas as pd

# --- util: serializa cualquier cosa (Zeep/dicts/listas) a estructuras nativas ---
def safe_serialize(obj):
    # tipos base
    if obj is None or isinstance(obj, (str, int, float, bool)):
        return obj
    if isinstance(obj, list):
        return [safe_serialize(x) for x in obj]
    if isinstance(obj, dict):
        return {k: safe_serialize(v) for k, v in obj.items()}

    # intentar con zeep.helpers.serialize_object (si está disponible)
    try:
        from zeep.helpers import serialize_object
        ser = serialize_object(obj)
        if ser is not None:
            return safe_serialize(ser)  # recursivo por si trae más objetos
    except Exception:
        pass

    # objetos Zeep suelen tener __values__
    vals = getattr(obj, "__values__", None)
    if isinstance(vals, dict):
        return {k: safe_serialize(v) for k, v in vals.items()}

    # fallback genérico
    try:
        return {k: safe_serialize(v) for k, v in vars(obj).items()}
    except Exception:
        return obj


def legislaturas_x_periodo(r):
    d = safe_serialize(r) or {}
    id_per = d.get("Id")

    # ¡OJO!: coalesce para evitar None.get(...)
    legis_node = d.get("Legislaturas") or {}
    legs = legis_node.get("Legislatura") or []

    # forzar lista
    if not isinstance(legs, list):
        legs = [legs]
    if not legs:
        return pd.DataFrame()

    # aplanar filas
    df = pd.json_normalize(legs, sep=".")

    # adjuntar Id del periodo
    df["Periodo_Id"] = id_per

    # renombrar 'Tipo'
    rename_map = {"Tipo._value_1": "Tipo_valor1", "Tipo.Valor": "Tipo_valor"}
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # tipar fechas (acepta strings o datetime de Python)
    for c in ("FechaInicio", "FechaTermino"):
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")

    return df


In [6]:
df_legislaturas_x_periodo = pd.concat(
    (legislaturas_x_periodo(r) for r in res),
    ignore_index=True
)

In [9]:
df_legislaturas_x_periodo.to_csv("legislaturas_x_periodo.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

In [10]:
df_legislaturas_x_periodo.columns

Index(['Id', 'Numero', 'FechaInicio', 'FechaTermino', 'Tipo_valor1',
       'Tipo_valor', 'Periodo_Id'],
      dtype='object')

### Mensajes por año

In [11]:
df_legislaturas_x_periodo["FechaInicio"] = pd.to_datetime(df_legislaturas_x_periodo["FechaInicio"], errors="coerce")
anio_min = df_legislaturas_x_periodo["FechaInicio"].dt.year.min()
anio_max = df_legislaturas_x_periodo["FechaInicio"].dt.year.max()

In [19]:
def get_msjs(year):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
    res = c.service.retornarMensajesXAnno(year)
    if res:
        d = safe_serialize(res) or {}
        df = pd.json_normalize(d, sep=".")
        return df

In [20]:
df_msjs = pd.concat((get_msjs(year) for year in range(anio_min, anio_max)))

<bound method NDFrame.describe of        Id NumeroBoletin                                             Nombre  \
0    1001        123-04  Establece condiciones de reprogramación para l...   
1    1017        214-10   Acuerdo relativo a la Convención Internaciona...   
2    1104        213-10   Acuerdo sobre la Convención Internacional con...   
3    1160        127-01  Permite a los pequeños agricultores recuperar ...   
4    1161         87-02  Modifica art. 61. ley de Reclutamiento y Movil...   
..    ...           ...                                                ...   
85  17523      16905-31  Reconoce el derecho al cuidado y crea el Siste...   
86  17738      17117-03  Reconoce y fortalece a las ferias libres como ...   
87  17445      16836-06  Modifica la ley N° 21.325 para perfeccionar el...   
88  17599      16985-06  Establece, para el personal de Gendarmería de ...   
89  17764      17142-05  Ley de Presupuestos para el Sector Público cor...   

   FechaIngreso Autores Minis

In [29]:
df_msjs.to_csv("mensajes.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

## Materias

In [30]:
c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
res = c.service.retornarMaterias()
res

[{
     'Id': 6656,
     'Nombre': 'JUICIOS DE FAMILIA'
 },
 {
     'Id': 5938,
     'Nombre': 'VIOLENCIA JUVENIL'
 },
 {
     'Id': 5940,
     'Nombre': 'MONTE FITZ ROY (ARGENTINA)'
 },
 {
     'Id': 5958,
     'Nombre': 'PESCA DEL JUREL'
 },
 {
     'Id': 5959,
     'Nombre': 'SISTEMA DE PUNTAJE'
 },
 {
     'Id': 5996,
     'Nombre': 'SUMINISTRO MÍNIMO DE USUARIOS'
 },
 {
     'Id': 5997,
     'Nombre': 'DESPIDO DEL TRABAJADOR'
 },
 {
     'Id': 5998,
     'Nombre': 'DIVORCIO UNILATERAL'
 },
 {
     'Id': 6157,
     'Nombre': 'CIRCUNSCRIPCIONES ELECTORALES'
 },
 {
     'Id': 6196,
     'Nombre': 'CULPABILIDAD'
 },
 {
     'Id': 6220,
     'Nombre': 'PROYECTOS INDUSTRIALES'
 },
 {
     'Id': 6221,
     'Nombre': 'JUREL'
 },
 {
     'Id': 6237,
     'Nombre': 'DIRECCION DEL TRABAJO'
 },
 {
     'Id': 6280,
     'Nombre': 'APARATOS DE SEGURIDAD\n'
 },
 {
     'Id': 6376,
     'Nombre': ' EJÉRCITO DE CHILE'
 },
 {
     'Id': 6617,
     'Nombre': 'INMIGRANTE ILEGAL'
 },
 {
     'Id': 661

In [31]:
d = safe_serialize(res) or {}
df_materias = json_normalize(d)
df_materias.head()

,Id,Nombre
0,6656,JUICIOS DE FAMILIA
1,5938,VIOLENCIA JUVENIL
2,5940,MONTE FITZ ROY (ARGENTINA)
3,5958,PESCA DEL JUREL
4,5959,SISTEMA DE PUNTAJE


In [32]:
df_materias.to_csv("materias.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

## Trámites constitucionales

In [33]:
c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
res = c.service.retornarTramitesConstitucionales()
res

[{
     '_value_1': 'Primer Trámite',
     'Id': 1
 },
 {
     '_value_1': 'Segundo Trámite',
     'Id': 2
 },
 {
     '_value_1': 'Tercer Trámite',
     'Id': 3
 },
 {
     '_value_1': 'Comisión Mixta',
     'Id': 4
 },
 {
     '_value_1': 'Observaciones P. De La R.',
     'Id': 5
 },
 {
     '_value_1': 'Único',
     'Id': 6
 }]

In [34]:
d = safe_serialize(res) or {}
df_tramites_constitucionales = json_normalize(d)
df_tramites_constitucionales.head()

,_value_1,Id
0,Primer Trámite,1
1,Segundo Trámite,2
2,Tercer Trámite,3
3,Comisión Mixta,4
4,Observaciones P. De La R.,5


In [35]:
df_tramites_constitucionales.to_csv("tramites_constitucionales.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

## Trámites reglamentarios

In [36]:
c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
res = c.service.retornarTramitesReglamentarios()
res

[{
     '_value_1': 'Primer Informe',
     'Id': 1
 },
 {
     '_value_1': 'Segundo Informe',
     'Id': 2
 },
 {
     '_value_1': 'Tercer Informe',
     'Id': 3
 },
 {
     '_value_1': 'Informe Observaciones',
     'Id': 4
 },
 {
     '_value_1': 'Primer Informe Complementario',
     'Id': 5
 },
 {
     '_value_1': 'Segundo Informe Complementario',
     'Id': 6
 },
 {
     '_value_1': 'Sin Informe',
     'Id': 7
 },
 {
     '_value_1': 'Informe Enmiendas Del Senado',
     'Id': 8
 }]

In [37]:
d = safe_serialize(res) or {}
df_tramites_reglamentarios = json_normalize(d)
df_tramites_reglamentarios.head()
df_tramites_reglamentarios.to_csv("tramites_reglamentarios.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

## Votaciones Por año

In [4]:
def get_vot_year(year):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
    res = c.service.retornarVotacionesXAnno(year)
    if res:
        d = safe_serialize(res) or {}
        df = json_normalize(d)
        return df

In [5]:
df_vot_year = pd.concat((get_vot_year(year) for year in range(2022, 2026)))

In [51]:
df_vot_year.to_csv("votaciones_year.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

In [9]:
df_vot_year["Fecha"].describe

<bound method NDFrame.describe of 0     2022-12-21 17:22:17
1     2022-12-21 13:17:46
2     2022-12-21 13:16:24
3     2022-12-21 13:15:06
4     2022-12-21 13:13:41
              ...        
963   2025-01-06 19:05:31
964   2025-01-06 17:29:59
965   2025-01-06 17:28:06
966   2025-01-06 17:26:13
967   2025-01-06 17:24:43
Name: Fecha, Length: 5980, dtype: datetime64[ns]>

## Detalle votaciones

In [6]:
def get_detalle(idx):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
    res = c.service.retornarVotacionDetalle(idx)
    if res:
        d = safe_serialize(res) or {}
        df = json_normalize(d)
        return df

In [10]:
df_detalle = pd.concat((get_detalle(idx) for idx in df_vot_year["Id"]))

In [11]:
df_explode = df_detalle.explode("Votos.Voto").reset_index(drop=True)

In [12]:
df_votos = json_normalize(df_explode["Votos.Voto"])

In [13]:
df_detalle_1 = pd.concat([df_explode, df_votos], axis=1)

In [15]:
df_detalle_1

,Id,Descripcion,Fecha,TotalSi,TotalNo,TotalAbstencion,TotalDispensado,Quorum._value_1,Quorum.Valor,Resultado._value_1,...,Diputado.ApellidoPaterno,Diputado.ApellidoMaterno,Diputado.FechaNacimiento,Diputado.FechaDefucion,Diputado.RUT,Diputado.RUTDV,Diputado.Sexo,Diputado.Militancias,OpcionVoto._value_1,OpcionVoto.Valor
0,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,Acevedo,Sáez,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
1,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,Ahumada,Palma,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
2,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,Alessandri,Vergara,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
3,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,Alinco,Bustos,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
4,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,Araya,Guerrero,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765971,52568,1-Otros,2025-01-06 17:24:43,63,26,11,0,Quórum Simple,1,Aprobado,...,Venegas,Salazar,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
765972,52568,1-Otros,2025-01-06 17:24:43,63,26,11,0,Quórum Simple,1,Aprobado,...,Von Mühlenbrock,Zamora,NaN,NaN,NaN,NaN,NaN,NaN,Abstención,2.0
765973,52568,1-Otros,2025-01-06 17:24:43,63,26,11,0,Quórum Simple,1,Aprobado,...,Weisse,Novoa,NaN,NaN,NaN,NaN,NaN,NaN,En Contra,0.0
765974,52568,1-Otros,2025-01-06 17:24:43,63,26,11,0,Quórum Simple,1,Aprobado,...,Winter,Etcheberry,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0


In [14]:
df_detalle_1.to_csv("detalle_voto.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

In [112]:
def get_votaciones_proyecto(idx):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
    res = c.service.retornarVotacionesXProyectoLey(idx)
    if res:
        d = safe_serialize(res) or {}
        df = json_normalize(d)
        return df

In [109]:
s_clean = (
    pd.Series(df_detalle_1.Descripcion.unique())
      .str.extract(r'(\d{3,6}(?:-\d{2})?)')[0] 
      .dropna()
)

print(s_clean)

0        2625-07
1        3107-05
4        2727-11
5        3039-07
6        3147-10
          ...   
5013    16720-15
5014    16754-06
5015        1337
5016        1336
5017    16433-18
Name: 0, Length: 4866, dtype: object


In [ ]:
c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
res = c.service.retornarVotacionesXProyectoLey("2625-07")
res

In [113]:
df_vot_proy = pd.concat((get_votaciones_proyecto(idx) for idx in s_clean))
df_vot_proy.head()

,Id,NumeroBoletin,Nombre,FechaIngreso,MinisteriosPatrocinantes,Admisible,TipoIniciativa._value_1,TipoIniciativa.Valor,CamaraOrigen._value_1,CamaraOrigen.Valor,Autores.ParlamentarioAutor,Votaciones.VotacionProyectoLey,Materias.Materia,Autores,MinisteriosPatrocinantes.Ministerio,Materias
0,1542,2625-07,Moderniza la normativa reguladora de los arren...,2000-11-15,None,True,Moción,2,Senado,2,"[{'Orden': 0, 'Diputado': None, 'Senador': {'I...","[{'Id': 15545, 'Descripcion': 'Boletín N°2625-...","[{'Id': 1917, 'Nombre': 'PREDIOS URBANOS'}]",NaN,NaN,NaN
0,3405,3107-05,Crea una bonificación a la contratación de man...,2002-10-17,NaN,True,Mensaje,1,Cámara de Diputados,1,NaN,"[{'Id': 14897, 'Descripcion': 'Boletín N°3107-...","[{'Id': 1608, 'Nombre': 'BONIFICACION POR CONT...",None,"[{'Id': 2, 'Nombre': 'Ministerio de Hacienda'}]",NaN
0,2140,2727-11,Sobre los derechos y deberes de las personas e...,2001-06-12,NaN,True,Mensaje,1,Cámara de Diputados,1,NaN,"[{'Id': 14893, 'Descripcion': 'Boletín N°2727-...","[{'Id': 1334, 'Nombre': 'SALIDA DE TROPAS'}]",None,"[{'Id': 52, 'Nombre': 'Ministerio de Salud'}]",NaN
0,3336,3039-07,Reforma constitucional que establece la obliga...,2002-09-03,None,True,Mensaje,1,Cámara de Diputados,1,NaN,"[{'Id': 15538, 'Descripcion': 'Boletín N°3039-...","[{'Id': 1542, 'Nombre': 'REFORMA CONSTITUCIONA...",None,NaN,NaN
0,3434,3147-10,Aprueba el acuerdo por el que se establece una...,2002-12-03,NaN,True,Mensaje,1,Cámara de Diputados,1,NaN,"[{'Id': 14213, 'Descripcion': 'Boletín N°3147-...","[{'Id': 19163, 'Nombre': 'ACUERDO DE ASOCIACIÓ...",None,"[{'Id': 51, 'Nombre': 'Ministerio de Relacione...",NaN


In [118]:
df_vot_proy.to_csv("votos_proyecto.csv", index=False, encoding="utf-8-sig", date_format="%Y-%m-%d")

In [ ]:
retornarProyectoLey(prmNumeroBoletin)
retornarProyectosLeyxNumeroLey(prmNumeroLey)